# Data analysis of the fantasy premier league

### Goal of Analysis: Use data science to get the best possible team for the next gameweek as well as suggest the best possible transfers according to the available resources.


In [80]:
import json
import numpy as np
import pandas as pd
import requests

In [81]:
with open('D:/programming/my_projects/fpl_ai_api_django/fpl_data.json','r',encoding='utf-8') as file:
    data=json.load(file)
    teams_df=pd.DataFrame(data['teams'])
    elements_df=pd.DataFrame(data['elements'])
    element_types_df=pd.DataFrame(data['element_types'])
    

In [91]:
teams_df.head()

,code,draw,form,id,loss,name,played,points,position,short_name,...,team_division,unavailable,win,strength_overall_home,strength_overall_away,strength_attack_home,strength_attack_away,strength_defence_home,strength_defence_away,pulse_id
0,3,0,None,1,0,Arsenal,0,0,0,ARS,...,None,False,0,1220,1270,1240,1250,1200,1270,1
1,7,0,None,2,0,Aston Villa,0,0,0,AVL,...,None,False,0,1090,1100,1110,1130,1090,1110,2
2,91,0,None,3,0,Bournemouth,0,0,0,BOU,...,None,False,0,1060,1090,1070,1130,1050,1080,127
3,94,0,None,4,0,Brentford,0,0,0,BRE,...,None,False,0,1100,1130,1100,1110,1130,1160,130
4,36,0,None,5,0,Brighton,0,0,0,BHA,...,None,False,0,1150,1160,1140,1160,1170,1190,131


In [82]:
elements_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 740 entries, 0 to 739
Data columns (total 88 columns):
 #   Column                                Non-Null Count  Dtype  
---  ------                                --------------  -----  
 0   chance_of_playing_next_round          503 non-null    float64
 1   chance_of_playing_this_round          501 non-null    float64
 2   code                                  740 non-null    int64  
 3   cost_change_event                     740 non-null    int64  
 4   cost_change_event_fall                740 non-null    int64  
 5   cost_change_start                     740 non-null    int64  
 6   cost_change_start_fall                740 non-null    int64  
 7   dreamteam_count                       740 non-null    int64  
 8   element_type                          740 non-null    int64  
 9   ep_next                               740 non-null    object 
 10  ep_this                               740 non-null    object 
 11  event_points       

In [83]:
slim_elements_df=elements_df[['second_name','element_type','team','total_points','minutes','selected_by_percent']]

In [84]:
slim_elements_df

,second_name,element_type,team,total_points,minutes,selected_by_percent
0,Xhaka,3,1,81,1706,2.8
1,Elneny,3,1,6,111,0.7
2,Holding,2,1,6,16,0.1
3,Partey,3,1,54,1450,0.5
4,Ødegaard,3,1,119,1598,35.0
...,...,...,...,...,...,...
735,Santos Carneiro Da Cunha,4,20,5,191,0.1
736,Lemina,3,20,5,165,0.0
737,Sarabia,3,20,2,104,0.0
738,Bentley,1,20,0,0,0.0


In [85]:
element_types_df

,id,plural_name,plural_name_short,singular_name,singular_name_short,squad_select,squad_min_play,squad_max_play,ui_shirt_specific,sub_positions_locked,element_count
0,1,Goalkeepers,GKP,Goalkeeper,GKP,2,1,1,True,[12],77
1,2,Defenders,DEF,Defender,DEF,5,3,5,False,[],250
2,3,Midfielders,MID,Midfielder,MID,5,2,5,False,[],321
3,4,Forwards,FWD,Forward,FWD,3,1,3,False,[],92


In [86]:
elements_df['position'] = elements_df.element_type.map(element_types_df.set_index('id').singular_name)

In [92]:
elements_df['team'] = elements_df.team.map(teams_df.set_index('id').name)

In [93]:
elements_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 740 entries, 0 to 739
Data columns (total 89 columns):
 #   Column                                Non-Null Count  Dtype  
---  ------                                --------------  -----  
 0   chance_of_playing_next_round          503 non-null    float64
 1   chance_of_playing_this_round          501 non-null    float64
 2   code                                  740 non-null    int64  
 3   cost_change_event                     740 non-null    int64  
 4   cost_change_event_fall                740 non-null    int64  
 5   cost_change_start                     740 non-null    int64  
 6   cost_change_start_fall                740 non-null    int64  
 7   dreamteam_count                       740 non-null    int64  
 8   element_type                          740 non-null    int64  
 9   ep_next                               740 non-null    object 
 10  ep_this                               740 non-null    object 
 11  event_points       

In [94]:
slim_elements_df=elements_df[['second_name','element_type','position','team','total_points','minutes','selected_by_percent']]

In [95]:
slim_elements_df

,second_name,element_type,position,team,total_points,minutes,selected_by_percent
0,Xhaka,3,Midfielder,Arsenal,81,1706,2.8
1,Elneny,3,Midfielder,Arsenal,6,111,0.7
2,Holding,2,Defender,Arsenal,6,16,0.1
3,Partey,3,Midfielder,Arsenal,54,1450,0.5
4,Ødegaard,3,Midfielder,Arsenal,119,1598,35.0
...,...,...,...,...,...,...,...
735,Santos Carneiro Da Cunha,4,Forward,Wolves,5,191,0.1
736,Lemina,3,Midfielder,Wolves,5,165,0.0
737,Sarabia,3,Midfielder,Wolves,2,104,0.0
738,Bentley,1,Goalkeeper,Wolves,0,0,0.0
